In [1]:
from preprocessing import *
from sklearn.model_selection import KFold
import argparse
from model import *
from train import test
import torch.optim as optim

from MatrixVectorizer import *


In [2]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = np.loadtxt(lr_data_path, delimiter=',')
hr_train_data = np.loadtxt(hr_data_path, delimiter=',')
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 260) for row in hr_train_data])
num_samples = hr_train_data_vectorized.shape[0]
split = int(num_samples * 0.8)

In [3]:

subjects_adj,subjects_labels = lr_train_data_vectorized[:split], hr_train_data_vectorized[:split]

held_out_subjects_adj,held_out_subjects_labels = lr_train_data_vectorized[split:], hr_train_data_vectorized[split:]

In [4]:
num_splt = 3
epochs = 15
lr = 0.001
lmbda = 16
lr_dim = 160
hr_dim = 320
hidden_dim = 320
padding = 30
dropout = 0
args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [5]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [6]:
# ks = [0]
ks = [0.7, 0.3]
model = GSRNet(ks, args)

In [7]:
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj,subjects_labels, args): 
  #, subjects_adj_test, subjects_ground_truth_test):
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []

    model.train()
    for lr,hr in zip(subjects_adj,subjects_labels):      
      lr = torch.from_numpy(lr).type(torch.FloatTensor)
      hr = torch.from_numpy(hr).type(torch.FloatTensor)
      
      
      # net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      # model_outputs  = unpad(model_outputs, args.padding)
      # weights = unpad(model.layer.weights, args.padding)
      

      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      # print the shapes of the outputs
      # print(f"{net_outs.shape} ; {start_gcn_outs.shape}")
      # print(f"{model.layer.weights.shape} ; {U_hr.shape}")
      # print(f"{model_outputs.shape} ; {hr.shape}")
      
      # loss = criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + args.lmbda * criterion(model_outputs, hr) 
      # loss = criterion(model_outputs, hr) 
      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, padded_hr) 

      
      error = criterion(model_outputs, padded_hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      
  
    model.eval()
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error))
    test(model, held_out_subjects_adj, held_out_subjects_labels, args)
    # test(model, subjects_adj_test, subjects_ground_truth_test, args)
    all_epochs_loss.append(np.mean(epoch_loss))

In [8]:
# print(model)
optimizer = optim.Adam(model.parameters(), lr=args.lr)
# optimizer = optim.SGD(model.parameters(), lr=args.lr)

train(model, optimizer, subjects_adj, subjects_labels, args)

print('Held out test score:')
test(model, held_out_subjects_adj, held_out_subjects_labels, args)
print('------------------------------')

Epoch:  1 Loss:  82.89344727770606 Error:  82.75224178173204
Epoch:  2 Loss:  82.82422540147803 Error:  82.73462091086071
Epoch:  3 Loss:  82.81352242760694 Error:  82.73449453005372
Epoch:  4 Loss:  82.80866831563301 Error:  82.73476081246983
Epoch:  5 Loss:  82.80508683299395 Error:  82.7347744709409
Epoch:  6 Loss:  82.80389080519107 Error:  82.73490901374772
Epoch:  7 Loss:  82.80273461764428 Error:  82.7352214969917
Epoch:  8 Loss:  82.80248434819393 Error:  82.73551215214738
Epoch:  9 Loss:  82.80234912938592 Error:  82.73551061982984
Epoch:  10 Loss:  82.80304316342321 Error:  82.73586775673859
Epoch:  11 Loss:  82.80261548241573 Error:  82.7359223379684
Epoch:  12 Loss:  82.80209098878636 Error:  82.73603570072063
Epoch:  13 Loss:  82.80149550402342 Error:  82.73593152759236
Epoch:  14 Loss:  82.80187646522006 Error:  82.73594717592445
Epoch:  15 Loss:  82.80088280007911 Error:  82.73597702478517
Held out test score:
Test error MAE:  0.17638734991059585
------------------------

In [9]:
# # print(model)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
# # optimizer = optim.SGD(model.parameters(), lr=args.lr)

# for train_index, test_index in cv.split(subjects_adj):
#     subjects_adj_train = subjects_adj[train_index]  # Get training data 
#     subjects_adj_test = subjects_adj[test_index]   # Get testing data 
#     subjects_ground_truth_train = subjects_labels[train_index]
#     subjects_ground_truth_test = subjects_labels[test_index]

#     train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args, subjects_adj_test, subjects_ground_truth_test)
    
#     print('Held out test score:')
#     test(model, held_out_subjects_adj, held_out_subjects_labels, args)
#     print('------------------------------')